<a href="https://colab.research.google.com/github/VietHaDuong/gomoku-player/blob/main/testagent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/sitfoxfly/gomoku-ai

In [ ]:
import re
import json
import random
from google.colab import userdata
from typing import Tuple
from gomoku import Agent
from gomoku.llm import OpenAIGomokuClient
from gomoku.core.models import Player, GameState, GameResult

class AgentG(Agent):

    def _setup(self):
      self.system_prompt = self._create_system_prompt()
      self.llm_client = OpenAIGomokuClient(api_key=userdata.get('omok'), model='Qwen/Qwen3-8B:featherless-ai', endpoint='https://router.huggingface.co/v1')

    def _create_system_prompt(self) -> str:
        """Create the system prompt that teaches the LLM how to play Gomoku."""
        return """
        You are an expert Gomoku (Five-in-a-Row) player. Your goal is to get 5 of your pieces
          in a row (horizontally, vertically, or diagonally) while preventing your opponent from doing the same.

          Strategic priorities:
          1. WIN: If you can make 5 in a row, do it immediately
          2. BLOCK: If opponent can make 5 in a row in the next move, block them immediately
          3. CREATE THREATS: Build sequences of 2-3 pieces to create multiple winning opportunities
          4. CONTROL CENTER: The center area is most valuable for creating multiple directions, try to play in the center area when there are opportunity
          5. CREATE FORKS: Try to create situations where you have multiple ways to win
          6. Extend strongest OPEN THREE (e.g XXX__ / _XXX_ / X_XX_ with at least one opening end)
          7. Block opponent's strongest OPEN THREE
          8. Prefer moves near recent stones (Chebyshev distance <= 2) and near center when strength is equal
          9. Never choose an occupied/out-of-bound cell

          You must respond with valid JSON in this exact format:
          {
              "reasoning": "Brief explanation of your strategic thinking",
              "row": <row_number>,
              "col": <col_number>
          }

          The row and col must be valid coordinates (0-indexed). Choose only empty positions marked with '.'.
          """.strip()

    async def get_move(self, game_state: GameState) -> Tuple[int, int]:
        print(f"\n🧠 {self.agent_id} is thinking...")

        try:

            board_str = game_state.format_board(formatter="standard")
            board_prompt = f"Current board state:\n{board_str}\n"
            board_prompt += f"Current player: {game_state.current_player.value}\n"

            # Create messages for the LLM
            messages = [
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": f"{board_prompt}\n\nPlease provide your next move as JSON."},
            ]

            print("💡 Full Prompt:\n\n")
            print(json.dumps(messages, indent=2, ensure_ascii=False))
            print()

            # Get response from LLM
            response = await self.llm_client.complete(messages)

            print("💡 Response:\n\n")
            print(response)
            print()

            if m := re.search(r"{[^}]+}", response, re.DOTALL):
                json_data = json.loads(m.group(0).strip())
                return json_data["row"], json_data["col"]

        except Exception as e:
            print(e)

        return self._get_fallback_move(game_state)

    def _get_fallback_move(self, game_state: GameState) -> Tuple[int, int]:
        return random.choice(game_state.get_legal_moves())